In [ ]:
#!code .

In [1]:
from Cryptocurrency_pricing.Market import deribit_data as dm
from Cryptocurrency_pricing.Models.utils import *
from Cryptocurrency_pricing.Models.BlackScholes import BlackScholes
from Cryptocurrency_pricing.Models.Merton import Merton
from Cryptocurrency_pricing.Models.Heston import Heston

import numpy as np
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go   

In [2]:
data = dm.Scraper(currency='BTC')

In [3]:
raw_df = data.collect_data(max_workers = 15, save_csv = False)

Data Collected


In [4]:
#raw_df.head()

In [5]:
BS = BlackScholes(raw_df.copy(), 'B')
M = Merton(raw_df.copy(), 'B')
H = Heston(raw_df.copy(), 'B')

In [7]:
BS.initialize(reset = True)
M.initialize(reset = True)
H.initialize(reset = True)

In [10]:
BS.append_imp_vol_to_df()
#M.append_imp_vol_to_df()
H.append_imp_vol_to_df()

In [11]:
BS.append_price()
#M.append_price()
H.append_price()

In [12]:
BS.df.head()

,S,K,V,_T,bids,asks,last_price,mark_price,option_type,mid_iv,mark_iv,mid,moneyness,I_VOL,IV_moneyness,BS_PRICE
330,34315.86,30000.0,8.2,0.000645,0.000000,0.001532,0.0005,0.000118,P,78.705,129.48,0.000766,1.143862,1.183667e+00,2.899115e+04,0.000765
181,34332.26,31000.0,52.0,0.000645,0.000500,0.001749,0.0005,0.000620,P,132.765,129.20,0.001124,1.107492,9.254924e-01,3.709621e+04,0.001123
45,34350.86,32000.0,24.3,0.000645,0.001567,0.003187,0.0025,0.002190,P,130.175,126.04,0.002377,1.073464,6.776612e-01,5.069032e+04,0.002373
243,34328.97,33000.0,43.8,0.000645,0.004831,0.006637,0.0060,0.005690,P,114.060,115.69,0.005734,1.040272,4.083210e-01,8.407348e+04,0.005719
33,34351.31,34000.0,20.0,0.000645,0.022581,0.025049,0.0220,0.023351,C,106.850,105.97,0.023815,1.010333,4.768374e-07,7.203988e+10,351.529184


In [13]:
H.df.head()

,S,K,V,_T,bids,asks,last_price,mark_price,option_type,mid_iv,mark_iv,mid,moneyness,I_VOL,IV_moneyness,HESTON_PRICE
330,34315.86,30000.0,8.2,0.000645,0.000000,0.001532,0.0005,0.000118,P,78.705,129.48,0.000766,1.143862,2097151.0,0.016363,-0.000589
181,34332.26,31000.0,52.0,0.000645,0.000500,0.001749,0.0005,0.000620,P,132.765,129.20,0.001124,1.107492,2097151.0,0.016371,0.000640
45,34350.86,32000.0,24.3,0.000645,0.001567,0.003187,0.0025,0.002190,P,130.175,126.04,0.002377,1.073464,2097151.0,0.016380,-0.000631
243,34328.97,33000.0,43.8,0.000645,0.004831,0.006637,0.0060,0.005690,P,114.060,115.69,0.005734,1.040272,2097151.0,0.016369,0.000623
33,34351.31,34000.0,20.0,0.000645,0.022581,0.025049,0.0220,0.023351,C,106.850,105.97,0.023815,1.010333,2097151.0,0.016380,0.012335


In [ ]:
#option.init_merton(m = 0.625038301534241, lam =0.0010591027301815744, v = 0.511097066109511,  )
option.init_merton(reset=True)
option.init_heston(reset=True)

In [ ]:
option.append_imp_vol_to_df()
option.append_loc_vol_to_df()

In [ ]:
option.append_BS_price()

In [ ]:
option.df.dropna(inplace=True)
option.df = option.df.drop(option.df[option.df.BS_PRICE >= 5].index) #a revoir
#option.df = option.df.drop(option.df[option.df.IV_moneyness >= .5*10**6].index)

In [ ]:
#option.optimize_merton(tol = 1e-6, max_iter = 600)



In [ ]:
option.append_Merton_price()
option.append_Heston_prices()

In [ ]:
x = option.df['S']/option.df['K']

#x = option.df['K']
y = option.df['_T']
z = option.df['L_VOL']
z2 = option.df['mark_iv']/100


fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Local vol')])
#fig = go.Figure(data=[go.Surface(x=x, y=y, z=z , name='Local vol')])
                      #go.Scatter3d(x=x, y=y, z=z2 , mode='markers', name='Implied vol deribit')])
"""
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price'),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price')])
"""

fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='IV') 
fig.show()

In [ ]:
#LINEAR INTERPOLATION
new_df = option.df[['moneyness', '_T', 'I_VOL_BS']]
new_df = new_df.pivot(index = '_T',columns='moneyness', values='I_VOL_BS')

new_df=new_df.interpolate(method='linear',axis=1,limit_direction='both')
z = new_df
x = list(new_df.columns)
y = list(new_df.index)


fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='Implied Volatility', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='IV') 
fig.show()

In [ ]:
#LINEAR INTERPOLATION
new_df = option.df[['moneyness', '_T', 'L_VOL']]
new_df = new_df.pivot(index = '_T',columns='moneyness', values='L_VOL')

new_df=new_df.interpolate(method='linear',axis=1,limit_direction='both')
z = new_df
x = list(new_df.columns)
y = list(new_df.index)


fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='Local Volatility', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.update_scenes(xaxis_title_text='S/K', yaxis_title_text='Exp', zaxis_title_text='Local Vol') 
fig.show()

In [ ]:

new_df = option.df[['K', '_T', 'L_VOL']]
new_df = new_df.pivot(index = '_T',columns='K', values='L_VOL')

new_df=new_df.interpolate(method='spline',order=2,axis=1)
z = new_df
x = list(new_df.columns)
y = list(new_df.index)


fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='xxx', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()


In [ ]:
#option.init_merton(m = 0.625038301534241, lam =0.0010591027301815744, v = 0.511097066109511,  )
option.init_merton(reset=True)
option.append_Merton_price()
rms = mean_squared_error(option.df['MERTON_PRICE'], option.df['mid'], squared=False)
print('Root Mean Squared Error = {}%'.format(round(rms*100,3)))

In [ ]:
option.df.dropna(inplace=True)
#option.df = option.df.drop(option.df[option.df._T >= .08].index)

In [ ]:
x = option.df['K']
y = option.df['_T']
z = option.df['BS_PRICE']
z2 = option.df['mid']
z3 = option.df['MERTON_PRICE']
z4 = option.df['HESTON_PRICE']


#fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='BS price')])

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z , mode='markers', name='Black&Scholes model Price', marker=dict(opacity=0.8)),
                      go.Scatter3d(x=x, y=y, z=z2, mode='markers', name='Real Market Price', marker=dict(opacity=0.8)),
                      go.Scatter3d(x=x, y=y, z=z3, mode='markers', name='Merton Model Price', marker=dict(opacity=0.8),),
                      go.Scatter3d(x=x, y=y, z=z4, mode='markers', name='Heston Model Price', marker=dict(opacity=0.8))])


fig.update_scenes(xaxis_title_text='Strike', yaxis_title_text='Exp', zaxis_title_text='Price') 
fig.show()

rms_merton = mean_squared_error(option.df['mid'], option.df['MERTON_PRICE'], squared=False)
rms_bs     = mean_squared_error(option.df['mid'], option.df['BS_PRICE'], squared=False)
rms_heston = mean_squared_error(option.df['mid'], option.df['HESTON_PRICE'], squared=False)

print('Root Mean Squared Error | MERTON = {} %'.format(round(rms_merton*100,6)))
print('Root Mean Squared Error | B&S    = {} %'.format(round(rms_bs*100,6)))
print('Root Mean Squared Error | HESTON    = {} %'.format(round(rms_heston*100,6)))

## Test optim Heston

In [ ]:
x0 = np.array([0.08, 0.10, 0.8, 3, 0.25])
res = option.optimize_heston(x0,tol = 1e-5, max_iter =300 )

In [ ]:
option.init_heston(theta = res)
option.append_Heston_prices()

In [ ]:
np.size(option.df['S'])

finir git
tests unitaires
heston + sauts